In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import desc, concat_ws

# Load the training data
training_data = spark.read.csv("Training data.csv", header=True, inferSchema=True)

# Load the test data
testing_data = spark.read.csv("Test data.csv", header=True, inferSchema=True)

# Create a unique ID for each user and product
user_indexer = StringIndexer(inputCol="user_id", outputCol="user_id_index").fit(training_data.union(testing_data))
product_indexer = StringIndexer(inputCol="Product_bought", outputCol="product_id_index").fit(training_data.union(testing_data))

# Convert the training data to a format that can be used for training
training_data = user_indexer.transform(training_data)
training_data = product_indexer.transform(training_data)
training_data = training_data.select("user_id_index", "product_id_index", "Order_value")

# Convert the test data to a format that can be used for testing
testing_data = user_indexer.transform(testing_data)
testing_data = product_indexer.transform(testing_data)
testing_data = testing_data.select("user_id_index", "product_id_index", "Order_value")

# Train an ALS model to predict the order value
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_index", itemCol="product_id_index", ratingCol="Order_value", coldStartStrategy="drop")
model = als.fit(training_data)

# Use the model to make recommendations for all users in the test data
user_recs = model.recommendForAllUsers(3)
user_recs = user_recs.select("user_id_index", "recommendations.product_id_index")

# Join the recommendation data with the product data to get the product categories
product_data = product_indexer.transform(training_data.select("product_id_index", "Product_bought"))
user_recs = user_recs.join(product_data, user_recs.product_id_index[0] == product_data.product_id_index, "left_outer")
user_recs = user_recs.select("user_id_index", "recommendations.Product_bought")

# Group the recommendations by user and concatenate them into a single column
user_recs = user_recs.groupBy("user_id_index").agg(concat_ws(",", "Product_bought").alias("recommendations"))

# Convert user_id_index back to user_id
user_recs = user_recs.join(user_indexer, user_recs.user_id_index == user_indexer.user_id_index, "left_outer")
user_recs = user_recs.select("user_id", "recommendations")

# Write the output to a CSV file
user_recs.write.mode("overwrite").csv("path/to/final.csv")
